In [1]:
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import operator
import collections

In [2]:
def compute_entropy(dataset):
    n = len(dataset)
    label_count = {}
    for data in dataset:
        temp = data[-1]
        if temp not in label_count.keys():
            label_count[temp] = 0
        label_count[temp] += 1
    Entropy = 0.0
    for key in label_count:
        prob = float(label_count[key])/n
        Entropy -= prob * math.log(prob,2)
    return Entropy

In [3]:
def split_dataset(dataset,axis,value):
    retDataset= []
    for feature in dataset:
        if feature[axis]==value:
            reducedvec=feature[:axis]
            reducedvec.extend(feature[axis+1:])
            retDataset.append(reducedvec)
    return retDataset

In [4]:
def choose(dataset):
    n = len(dataset[0])-1
    baseentroy = compute_entropy(dataset)
    bestinfogain =0.0
    bestfeature = -1
    for i in range(n):
        choice = [example[i] for example in dataset]
        uni_val = set(choice)
        entro = 0.0
        for value in uni_val:
            split_data = split_dataset(dataset,i,value)
            p = len(split_data)/float(len(dataset))
            entro += p*compute_entropy(split_data)
        info_gain = baseentroy - entro
        if info_gain > bestinfogain:
            bestinfogain = info_gain
            bestfeature = i
    return bestfeature

In [5]:
def find_max(data):
    count = {}
    for vote in data:
        if vote in count.keys():
            count[vote] += 1
        elif vote not in count.keys():
            count[vote] = 1
        sorted_count = sorted(count.iteritems(),key = operator.itemgetter(1),reverse = True)
    return sorted_count[0][0]

In [6]:
def Create_Tree(dataset,labels):
    classlist = [exp[-1] for exp in dataset]
    if classlist.count(classlist[0]) == len(classlist):
        return classlist[0]
    if len(dataset[0]) == 1:
        return find_max(classlist)
    bestfeat = choose(dataset)
    bestlabel = labels[bestfeat]
    Tree = {bestlabel:{}}
    del(labels[bestfeat])
    feat_val = [exp[bestfeat] for exp in dataset]
    uni_val = set(feat_val)
    for val in uni_val:
        sublabel = labels[:]
        Tree[bestlabel][val] = Create_Tree(split_dataset(dataset,bestfeat,val),sublabel)
    return Tree

In [7]:
def createPlot(inTree):
    fig = plt.figure(1,facecolor='white')
    fig.clf()
    axprops = dict(xticks=[],yticks = [])
    createPlot.ax1 = plt.subplot(111,frameon = False, **axprops)
    plotTree.totalW = float(getNumleafs(inTree))
    plotTree.totalD = float(getTreeDepth(inTree))
    plotTree.xOff = -0.5/plotTree.totalW
    plotTree.yOff = 1.0
    plotTree(inTree,(0.5,1.0),'')
    plt.show()

In [8]:
def plotNode(nodeText, centerPt,parentPt,nodeType):
    createPlot.ax1.annotate(nodeText, xy = parentPt,xycoords = 'axes fraction',\
                           xytext = centerPt, textcoords = 'axes fraction',\
                           va = "center",ha = "center", bbox=nodeType,arrowprops= arrow_args)

In [9]:
def getNumleafs(myTree):
    num = 0
    first_str = list(myTree.keys())[0]
    secondDict = myTree[first_str]
    for key in secondDict.keys():
        if type(secondDict[key]).__name__ == 'dict':
            num = num + getNumleafs(secondDict[key])
        else:
            num = num + 1
    return num

In [10]:
def getTreeDepth(myTree):
    maxDepth = 0
    Depth = 0
    first_str = list(myTree.keys())[0]
    secondDict = myTree[first_str]
    for key in secondDict.keys():
        if type(secondDict[key]).__name__ == 'dict':
            Depth = 1 + getTreeDepth(secondDict[key])
        else:
            Depth += 1
        if Depth > maxDepth:
            maxDepth = Depth
    return maxDepth

In [11]:
def plotMidText(cntrPt, parentPt,txtString):
    xMid = (parentPt[0]-cntrPt[0])/2.0 + cntrPt[0]
    yMid = (parentPt[1]-cntrPt[1])/2.0 + cntrPt[1]
    createPlot.ax1.text(xMid,yMid,txtString)

In [12]:
def plotTree(myTree,parentPt,nodeText):
    num_of_leaves = getNumleafs(myTree)
    depth = getTreeDepth(myTree)
    firstStr = list(myTree.keys())[0]
    cntrPt = (plotTree.xOff + (1.0 + float(num_of_leaves))/2.0/plotTree.totalW,\
              plotTree.yOff)
    plotMidText(cntrPt,parentPt,nodeText)
    plotNode(firstStr,cntrPt,parentPt,decisionNode)
    secondDict = myTree[firstStr]
    plotTree.yOff = plotTree.yOff - 1.0/plotTree.totalD
    for key in secondDict.keys():
        if type(secondDict[key]).__name__ =='dict':
            plotTree(secondDict[key],cntrPt,str(key))
        else:
            plotTree.xOff = plotTree.xOff + 1.0/plotTree.totalW
            plotNode(secondDict[key],(plotTree.xOff,plotTree.yOff),cntrPt,leafNode)
            plotMidText((plotTree.xOff,plotTree.yOff),cntrPt,str(key))
    plotTree.yOff = plotTree.yOff + 1.0/plotTree.totalD

In [13]:
iris = sklearn.datasets.load_iris()
labels = iris.feature_names
n = iris.data.shape[0]
index = [i for i in range(n)]
random.shuffle(index) 
iris.data = iris.data[index]
iris.target = iris.target[index]
iris.data = np.insert(iris.data,np.size(iris.data,1),values= iris.target,axis = 1)
dataset = iris.data.tolist()
decisionNode = dict(boxstyle = "sawtooth",fc = "0.8")
leafNode = dict(boxstyle = "round4",fc = "0.8")
arrow_args = dict(arrowstyle = "<-")

In [14]:
Decision_Tree = Create_Tree(dataset,labels)
Decision_Tree

{'petal length (cm)': {1.4: 0.0,
  1.6: 0.0,
  3.3: 1.0,
  4.2: 1.0,
  4.5: {'sepal length (cm)': {4.9: 2.0,
    5.4: 1.0,
    6.4: 1.0,
    6.0: 1.0,
    6.2: 1.0,
    5.7: 1.0,
    5.6: 1.0}},
  6.9: 2.0,
  5.0: {'sepal length (cm)': {6.7: 1.0, 5.7: 2.0, 6.0: 2.0, 6.3: 2.0}},
  5.1: {'sepal length (cm)': {5.8: 2.0,
    6.3: 2.0,
    6.5: 2.0,
    6.0: 1.0,
    6.9: 2.0,
    5.9: 2.0}},
  4.3: 1.0,
  4.9: {'sepal width (cm)': {2.5: 1.0,
    3.0: 2.0,
    3.1: 1.0,
    2.8: 2.0,
    2.7: 2.0}},
  1.5: 0.0,
  6.1: 2.0,
  4.8: {'sepal length (cm)': {6.0: 2.0, 5.9: 1.0, 6.2: 2.0, 6.8: 1.0}},
  5.6: 2.0,
  6.6: 2.0,
  3.0: 1.0,
  3.5: 1.0,
  4.0: 1.0,
  4.4: 1.0,
  4.6: 1.0,
  4.1: 1.0,
  5.9: 2.0,
  5.4: 2.0,
  5.5: 2.0,
  6.0: 2.0,
  6.4: 2.0,
  1.0: 0.0,
  1.1: 0.0,
  3.6: 1.0,
  1.2: 0.0,
  1.7: 0.0,
  3.7: 1.0,
  3.8: 1.0,
  4.7: 1.0,
  1.3: 0.0,
  5.2: 2.0,
  5.8: 2.0,
  5.7: 2.0,
  5.3: 2.0,
  6.3: 2.0,
  6.7: 2.0,
  1.9: 0.0,
  3.9: 1.0}}

In [ ]:
createPlot(Decision_Tree)

###### 数据可视化部分学习并参考了《机器学习实战》的代码